In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
import pandas as pd
df = pd.read_csv('/home/babycj/Complex_System_Journal_Final_code/data/monks-3-new.csv')

In [3]:
y=df['Class']
df_with_class=df.copy()
df.drop(['Class'],axis=1,inplace=True)
X=df

In [4]:
dict_x=X.to_dict()

In [5]:
from math import ceil
from math import floor

import pandas as pd



def check_type(variable):
    if isinstance(variable, (int, float)):
        return "Number"
    elif isinstance(variable, str):
        return "String"
    else:
        return "Other type"




def frequency_encoding(data):
    bits = 3
    encoder = ['000', '001', '011', '111']  # for numeric encoding

    # Define categorical encodings
    categorical_map = {
        "yesno": {"yes": "01", "no": "10"},
        "lowhigh": {"low": "01", "high": "10"},
        "lowmedhigh": {"low": "001", "medium": "010", "high": "100"}
    }

    for col in data:
        first_val = str(data[col][0]).strip().lower()

        # --- Handle Yes/No ---
        if first_val in ["yes", "no"]:
            for i, val in data[col].items():
                val_str = str(val).strip().lower()
                data[col][i] = categorical_map["yesno"].get(val_str, "00")

        # --- Handle Low/Medium/High ---
        elif first_val in ["low", "high"]:
            for i, val in data[col].items():
                val_str = str(val).strip().lower()
                data[col][i] = categorical_map["lowhigh"].get(val_str, "00")

        elif first_val in ["low", "medium", "high"]:
            for i, val in data[col].items():
                val_str = str(val).strip().lower()
                data[col][i] = categorical_map["lowmedhigh"].get(val_str, "000")

        # --- Handle numeric/string frequency encoding ---
        elif check_type(data[col][0]) in ["String", "Number"]:
            col_arr = list(data[col].values())
            col_arr.sort()
            t = int(len(col_arr) / (bits + 1))
            r = len(col_arr) % (bits + 1)
            index_map = {}
            j = 0
            for i in range(len(col_arr)):
                if (i + 1) % t == 0:
                    index_map[col_arr[i]] = j
                    j += 1
                    j = min(j, 3)
                else:
                    index_map[col_arr[i]] = j
            for i, (key, value) in enumerate(data[col].items()):
                data[col][i] = encoder[index_map[value]]

    return data

In [6]:
data=frequency_encoding(dict_x)

In [7]:
X=pd.DataFrame(data)
y=LabelEncoder().fit_transform(y)

In [8]:
X['Target Configuration']=X.apply(lambda row: ''.join(map(str, row)), axis=1)

In [9]:
X

,A1,A2,A3,A4,A5,A6,Target Configuration
0,001,001,001,001,000,001,001001001001000001
1,001,001,001,001,000,111,001001001001000111
2,001,001,001,001,001,001,001001001001001001
3,001,001,001,001,001,111,001001001001001111
4,001,001,001,001,011,001,001001001001011001
...,...,...,...,...,...,...,...
427,111,111,111,111,001,111,111111111111001111
428,111,111,111,111,011,001,111111111111011001
429,111,111,111,111,011,111,111111111111011111
430,111,111,111,111,111,001,111111111111111001


In [10]:
# import pandas as pd
# Rename columns in X
X = X.rename(columns={
    "Feature_1": "Encoded_Feature_1",
    "Feature_2": "Encoded_Feature_2"
})
# Reset indexes before concatenation to avoid NaN
df_final = pd.concat([df.reset_index(drop=True),
                      X.reset_index(drop=True)],
                      axis=1)

# Add target as last column
df_final["target"] = y

print(df_final.to_string(line_width=200))
# print(df_final)

    A1 A2 A3 A4 A5 A6   A1   A2   A3   A4   A5   A6 Target Configuration  target
0    1  1  1  1  1  1  001  001  001  001  000  001   001001001001000001       1
1    1  1  1  1  1  2  001  001  001  001  000  111   001001001001000111       1
2    1  1  1  1  2  1  001  001  001  001  001  001   001001001001001001       1
3    1  1  1  1  2  2  001  001  001  001  001  111   001001001001001111       1
4    1  1  1  1  3  1  001  001  001  001  011  001   001001001001011001       1
5    1  1  1  1  3  2  001  001  001  001  011  111   001001001001011111       1
6    1  1  1  1  4  1  001  001  001  001  111  001   001001001001111001       0
7    1  1  1  1  4  2  001  001  001  001  111  111   001001001001111111       0
8    1  1  1  2  1  1  001  001  001  011  000  001   001001001011000001       1
9    1  1  1  2  1  2  001  001  001  011  000  111   001001001011000111       1
10   1  1  1  2  2  1  001  001  001  011  001  001   001001001011001001       1
11   1  1  1  2  2  2  001  

In [11]:
import math
def lcm(a, b):
    return abs(a * b) // math.gcd(a, b)

def pad_configurations(training_data, n):
    padded_data = {}
    #target_multiple = lcm(n, 3)  # Make length a multiple of both n and 3
    target_multiple = n
    for config, label in training_data.items():
        length = len(config)
        remainder = length % target_multiple
        if remainder != 0:
            padding = target_multiple - remainder
            config = '0' * padding + config  # prepend zeros
        padded_data[config] = label
    # Print length of one padded config
    example_config = next(iter(padded_data))
    #print("length of padded data =", len(example_config))
    return padded_data

In [12]:
# import math

# def find_optimal_n(training_data):
#     lengths = [len(config) for config in training_data]
#     max_len = max(lengths)

#     # If configurations are small, no need to compute optimal n
#     if max_len <= 21:
#         return max_len

#     # Else, find optimal n
#     for n in range(1, 21):
#         lcm_val = abs(n * 3) // math.gcd(n, 3)
#         total_chunks = 0

#         for config in training_data:
#             length = len(config)
#             padded_length = length
#             if length % lcm_val != 0:
#                 padding = lcm_val - (length % lcm_val)
#                 padded_length += padding
#             total_chunks += padded_length // n

#         if total_chunks < 2**n:
#             return n  # Found smallest n satisfying condition

#     return None  # No suitable n found

In [13]:
import math

def find_optimal_n(training_data):
    if not training_data:
        return None  # no data
    
    l = len(next(iter(training_data)))   # length of any one config
    num_instances = len(training_data)
    #print("length of configuration",l)
    for s in range(2, 22):  # candidate split sizes 1..21
        chunks = math.ceil(l / s)             # chunks per config
        #print("Segments",chunks, "each with split size ",s)
        total_chunks = chunks * num_instances # total across all configs
        #print("Before split number of instances",num_instances)
        #print("After split number of instances",total_chunks)
        # Condition
        if total_chunks < 2 ** s:
            #print("Satisfied n is",s)
            return s
        else:
            #print("Split size not optimal",s)
            print("\n")
            
           

    return None  # No suitable split size found


In [14]:
import random
def solve(number, rulenumber):
    if ((1 << number) & rulenumber) > 0:
        return 1
    return 0

# m = {
#     0: [(12, 1), (3, 1), (6, 3), (9, 3)],
#     1: [(204, 1), (51, 1),(85, 2),(54, 5),(57, 5), (99, 5), (108, 5), (147, 5), (156, 5), (198, 5), (201, 5)],
#     2: [(15, 1)],
#     3: [ (204, 1), (85, 2), (27, 5),(39, 5), (78, 5), (114, 5), (141, 5), (177, 5), (216, 5), (228, 5)],
#     5: [(51, 1), (204, 1), (85, 2)]

# }
# endround = {
#     1: [ 20, 65, 68],
#     2: [5, 20, 65, 80],
#     3: [5, 68, 80],
#     4: [20, 65],
#     5: [ 68],

# }



m = {
    0: [(3, 1), (12, 1), (6, 3), (9, 3)],
    1: [(51, 1), (204, 1), (60, 1), (195, 1), (102, 3), (153, 3),
        (53, 4), (58, 4), (83, 4), (92, 4), (163, 4), (172, 4), (197, 4), (202, 4), (54, 5), (57, 5), (99, 5), (108, 5),
        (147, 5), (156, 5), (198, 5), (201, 5)],
    3: [(51, 1), (204, 1), (102, 3), (153, 3),
        (23, 4), (43, 4), (77, 4), (113, 4), (142, 4), (178, 4), (212, 4), (232, 4), (27, 5), (39, 5), (78, 5), (114, 5),
        (141, 5), (177, 5), (216, 5), (228, 5)],
    4: [(60, 1), (195, 1)],
    5: [(51, 1), (204, 1), (102, 3), (153, 3)]
}



endround = {
    1: [17, 20, 65, 68],
    3: [17, 68],
    4: [20, 65],
    5: [17, 68],
}
def rule_to_binary(rule, num_states):
    return format(rule, '0' + str(num_states**3) + 'b')

def evolve_cell(left, center, right, rule_number):
    index = int(str(left)+str(center)+str(right), 2)
    if((1<<index)&rule_number)>0:
        return '1'
    return '0'

def evolve_ca(row, rule_numbers):
    num_cells = len(row)
    next_row =""

    for i in range(num_cells):
        if(i-1<0):
            left=0
        else:
            left = int(row[(i - 1) % num_cells])
        center = int(row[i])
        if(i+1==num_cells):
            right=0
        else:
            right = int(row[(i + 1) % num_cells])
        next_row+= evolve_cell(left, center, right, rule_numbers[i])
    return next_row

def binary_to_decimal(binary):
    decimal=int(binary,2)
    return decimal

def decimal_to_binary(number,n):
    binary=bin(number)
    binary=binary[2:]
    if len(binary) is not n:
        for i in range(n-len(binary)):
            binary='0'+binary
    return binary

def binatodeci(binary):
    return int(binary,2)

def generate_ca(number, rule_numbers, all_numbers,n):
    initial_row=decimal_to_binary(number,n)
    ca_history = [initial_row]
    #print("Intial configuration",initial_row)
    s=set()
    all_numbers[number]=1
    while True:
        new_row = evolve_ca(ca_history[-1], rule_numbers)
        #print("Next configuration",new_row)
        if new_row in s:
            return -1
        if new_row==initial_row:
            ca_history.append(new_row)
            break
        s.add(new_row)
        ca_history.append(new_row)
        # print(new_row)
        number=binatodeci(new_row)
        all_numbers[number]=1
    return ca_history

def get_index(arr):
    return arr[1]

def classification(data,testing_data,n_op,rule_numbers):
    #print("Cell Length:",n)
    t=2**n_op



    all_numbers=[0]*t
    ca_history=[]
    for i in range(t):
        if(all_numbers[i]==0):
            history = generate_ca(i, rule_numbers,all_numbers,n_op)
            #print("cycles generating:",history)
            ca_history.append(history)
            if history==-1:
                ca_history=[]
                break

    map_to_cycle={}

    #print(ca_history)
    #Number_of_cycles=len(ca_history)
    # print(Number_of_cycles)
    # Length of ca_history
    xor_values_for_each_cycle = []   # Stores [cycle_index, xor_value] for each cycle

    for i in range(len(ca_history)):   # Iterate over each cycle in the CA history
        xor_label_index_dict = []      # Temporary list to hold [cycle_index, xor_value]
        xor_a = 0                      
        
        for j in range(len(ca_history[i])):   # Iterate over each configuration in cycle i
            num = binary_to_decimal(ca_history[i][j])   # Convert binary configuration to decimal
            xor_a = xor_a ^ num                         # Update XOR value with current 
            map_to_cycle[ca_history[i][j]] = i          # Map configuration to its cycle index
        
        xor_label_index_dict.append(i)      # Add cycle index
        xor_label_index_dict.append(xor_a)  # Add final XOR value for this cycle
        
        xor_values_for_each_cycle.append(xor_label_index_dict)  # Store result

    # print(1)

    # First, find number of classes (maximum label + 1)
    num_classes = max(data.values()) + 1

    # Initialize count dictionary with zeroes for all classes
    count_majority_for_each_cycle = {}
    for i in range(len(ca_history)):
        count_majority_for_each_cycle[i] = [0] * num_classes
    #print("Count_Majority in cycle",count_majority_for_each_cycle)

    # Count how many configs of each label fall in each cycle
    #print("data",data)

    #print("Cycle:",map_to_cycle)
    for config, label in data.items():
        #print("config and cycle config",map_to_cycle)
        cycle_index = map_to_cycle[config]
        count_majority_for_each_cycle[cycle_index][label] += 1
    #print("Count_Majority in cycle",count_majority_for_each_cycle)
    #print("Rule:", rule_numbers)
    cycle_label={}
    # print("\nAssigned cycle labels:")
    # for cycle_id, label in cycle_label.items():
    #     print(f"Cycle {cycle_id}: Label = {label}")
    for i, (key, value) in enumerate(count_majority_for_each_cycle.items()):  
        # i = index of the cycle  
        # key = cycle id (e.g., 0, 1, 2, ...)  
        # value = list of counts of class labels for this cycle  

        label = value.index(max(value))  
        # Find the label with the maximum count (majority label)  

        c = 0  
        for j in range(len(value)):  
            if (value[label] == value[j]):  
                c += 1  
        # Count how many labels have the same max frequency (to detect ties)  

        # Generate a unique conflict label ID  
        conflict_label = max(  
            set(label for label_list in count_majority_for_each_cycle.values()  
                for label in range(len(label_list)))  
        ) + 1  
        # This ensures conflict labels are different from normal labels  

        if c > 1:  
            # If there is a tie (multiple labels with same max count)  
            cycle_label[i] = conflict_label  
            xor_values_for_each_cycle[i].append(conflict_label)  
        else:  
            # Otherwise, assign the majority label normally  
            cycle_label[i] = label  
            xor_values_for_each_cycle[i].append(label)  


    # STEP 1: Sort based on some feature (XOR values )
    sorted_array = sorted(xor_values_for_each_cycle, key=get_index)
    #print("Rule:", rule_numbers)

    
    # STEP 2: Identify large gaps in index values to segment clusters
    gap_index_arr = [0]  
    # Stores the indices where a "gap" is detected in sorted_array
    # Start with 0 (beginning of array)

    curr_diff = sorted_array[1][1] - sorted_array[0][1]  
    # Initial difference between first two values (second column of sorted_array)

    for i in range(2, len(sorted_array)):  
        diff = sorted_array[i][1] - sorted_array[i-1][1]  
        # Compute difference between consecutive values

        if curr_diff == 0:  
            curr_diff = diff  
            # If current difference is 0, update it with new diff (to avoid divide-by-zero)

        elif diff > 10 * curr_diff:  
            # If the new difference is much larger (10x threshold),  
            # treat this as a "gap" → mark index i  
            gap_index_arr.append(i)  
            curr_diff = 0  
            # Reset curr_diff so next comparison starts fresh  
            continue  

        curr_diff = diff  
        # Update current difference for next iteration

    gap_index_arr.append(len(sorted_array))  
# Always add the last index (end boundary)




    # STEP 3: Count labels in each cluster and resolve conflict (label == conflict_label)
    conflict_label = max([x[2] for x in sorted_array])  # assumed to be the last label (e.g., if k=4, conflict_label=4)
    num_classes = conflict_label  # excluding the conflict label

    j = 1
    label_count = [0] * num_classes  # dynamically sized
    for i in range(len(sorted_array) + 1):
        if i == gap_index_arr[j]:
            # determine the label with max frequency
            max_val = max(label_count)
            tied_labels = [i for i, count in enumerate(label_count) if count == max_val]

            # Check if tie
            if len(tied_labels) == 1:
                max_label = tied_labels[0]
            else:
                # Tie-break: use training data present in this cluster
                class_counts_in_cluster = {label: 0 for label in tied_labels}

                # Iterate through cycles in current cluster
                for k in range(gap_index_arr[j-1], gap_index_arr[j]):
                    cycle_id = sorted_array[k][0]
                    if cycle_id in count_majority_for_each_cycle:
                        for label in tied_labels:
                            class_counts_in_cluster[label] += count_majority_for_each_cycle[cycle_id][label]

                # Choose the label with highest count from training data
                max_label = max(class_counts_in_cluster, key=class_counts_in_cluster.get)

            # resolve conflicts in this cluster
            for k in range(gap_index_arr[j-1], gap_index_arr[j]):
                if sorted_array[k][2] == conflict_label:
                    cycle_id = sorted_array[k][0]
                    cycle_label[cycle_id] = max_label
                    sorted_array[k][2] = max_label
            # reset counters for next cluster
            label_count = [0] * num_classes
            j += 1

        if i < len(sorted_array):
            label = sorted_array[i][2]
            if label != conflict_label:
                label_count[label] += 1




    Cycle_representative_class_label = {}

    # pick first configuration from each cycle as representative
    for cycle_id, label in cycle_label.items():
        if cycle_id < len(ca_history) and ca_history[cycle_id]:
            representative_config = ca_history[cycle_id][0]  # first config in cycle
            Cycle_representative_class_label[representative_config] = label

    #print("Cycle_representative_class_label:", Cycle_representative_class_label)
   
   
   
    from sklearn.metrics import precision_score, recall_score, f1_score

# --- Testing ---
    count = 0
    y_true = []
    y_pred = []

    for i, (config, true_label) in enumerate(testing_data.items()):
        # Step 1: Split the configuration into n-sized chunks
        chunks = [config[j:j+n_op] for j in range(0, len(config), n_op)]
        chunk_labels = []

        for chunk in chunks:
            start_config = chunk
            current_config = chunk
            predicted = None

            while True:
                # evolve one step
                next_config = evolve_ca(current_config, rule_numbers)

                # check if next config has a representative label
                if next_config in Cycle_representative_class_label:
                    predicted = Cycle_representative_class_label[next_config]
                    break

                # stop if we came back to start (no representative found)
                if next_config == start_config:
                    break

                # move forward
                current_config = next_config

            if predicted is not None:
                chunk_labels.append(predicted)

        # Step 2: Majority vote from chunk labels
        if chunk_labels:
            predicted_label = max(set(chunk_labels), key=chunk_labels.count)

            # Step 3: Compare with true label
            y_true.append(true_label)
            y_pred.append(predicted_label)

            if predicted_label == true_label:
                count += 1

    # --- Metrics ---
    accuracy = count / len(y_true) if y_true else 0.0
    precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
    recall = recall_score(y_true, y_pred, average="macro", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)

    # --- Results ---
    result = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "rule_numbers": rule_numbers.copy(),
        "Number_of_cycles": len(ca_history),
        "Cycle_lengths": [len(inner) - 1 for inner in ca_history]
    }

    return result



In [15]:
from sklearn.model_selection import KFold
import random
import time

from sklearn.model_selection import KFold
import random
import time
top_10_runs = []  # Will only ever store 10 best runs
# rule_numbers = []
max_ans=0  
for times in range(1):
      initial_class = 0
      prev_rule = None
      
      # Generate rule sequence

      import pandas as pd
      #print("Data",X,y)
      y = pd.Series(y)  # convert y to Pandas Series
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
      # Convert training data into dictionary
      #print("Training data X_train",X_train,y_train)
      X_train = X_train.to_dict()
      training_data = {}
      count = 0
      #print("Training data X_train",X_train,y_train)
      for i, (key, value) in enumerate(X_train['Target Configuration'].items()):
          if value in training_data:
              if training_data[value] != y_train.iloc[i]:
                  count += 1
          training_data[value] = y_train.iloc[i]
      #print(("Training data before padding",training_data))
      # Find optimal n and pad configs
      n_opt = find_optimal_n(training_data)
      #n=n_opt
      print("Optimal n:", n_opt)
      
      rule_numbers = []
      while True:  # keep regenerating until condition satisfied
        rule_numbers.clear()
        initial_class=0
        for i in range(n_opt - 1):
                # Condition 1: if prev_rule is 60 or 195 → choose from (102, 3), (153, 3)
                if prev_rule in [60, 195]:
                    random_num = random.choice([(102, 3), (153, 3)])
                # Normal case
                else:
                    random_num = random.choice(m[initial_class])

                rule_numbers.append(random_num[0])
                initial_class = random_num[1]
                prev_rule = random_num[0]

            # Final rule selection
        random_num = random.choice(endround[initial_class])
        rule_numbers.append(random_num)
        if 60 in rule_numbers or 195 in rule_numbers:
            break   # accept this set
      #print(("Training data before padding",training_data))
      
      rule_numbers=[12, 172, 195, 102, 142, 195, 102, 178, 195, 68]
      print("Rule sequence:", rule_numbers)
      # if n_opt >= 22:
      training_data = pad_configurations(training_data, n_opt)
      #print(("Training data after padding",training_data))
      # Create chunked training data
      new_training_data = {}
      for config, label in training_data.items():
          for i in range(0, len(config), n_opt):
              chunk = config[i:i + n_opt]
              new_training_data[chunk] = label


      testing_data = {}
      for i, (key, value) in enumerate(X_test['Target Configuration'].items()):
        testing_data[value] = y_test.iloc[i]

      # ✅ Pad testing data (same as training)
      testing_data = pad_configurations(testing_data, n_opt)
      #print("Testing data after padding:", testing_data)

      
      # Run classification and track time
      start_time = time.time()
      #print("Training and testing data send",new_training_data,testing_data)
      result = classification(new_training_data, testing_data, n_opt, rule_numbers)
      end_time = time.time()
      execution_time = end_time - start_time
      # print(new_training_data)
      # print(testing_data)
      # print(f"Fold {fold+1} accuracy: {result['accuracy']:.4f}")
      print(f"Execution time: {execution_time:.2f} seconds")
      print(f"Accuracy : {result['accuracy']:.4f}")
      print(f"Precision: {result['precision']:.4f}")
      print(f"Recall   : {result['recall']:.4f}")
      print(f"F1-Score : {result['f1_score']:.4f}")
      if(max_ans<result['accuracy']):
          max_ans=result['accuracy']
      print("\n")
      with open("monks-3-new_results.txt", "a") as f:
        f.write(f"Rule sequence: {rule_numbers}, accuracy #: {result['accuracy']:.4f}\n")
        # f.write("\n")
      print("Maximum Accuracy=",max_ans)


















Optimal n: 10
Rule sequence: [12, 172, 195, 102, 142, 195, 102, 178, 195, 68]
Execution time: 0.06 seconds
Accuracy : 0.8851
Precision: 0.9057
Recall   : 0.8864
F1-Score : 0.8838


Maximum Accuracy= 0.8850574712643678


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

# --- Split dataset (keep as DataFrame/array, not dict!) ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Models ---
models = {
    "SVM": SVC(kernel="rbf", random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "MLP": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
}

# --- Training & Evaluation ---
results = {}

for name, model in models.items():
    model.fit(X_train, y_train)  # ✅ Works now (X_train is numeric DataFrame/array)
    y_pred = model.predict(X_test)

    results[name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, average="macro", zero_division=0),
        "Recall": recall_score(y_test, y_pred, average="macro", zero_division=0),
        "F1-score": f1_score(y_test, y_pred, average="macro", zero_division=0),
    }

# --- Print Results ---
for name, metrics in results.items():
    print(f"\n{name} Results:")
    print(f"  Accuracy : {metrics['Accuracy']:.4f}")
    print(f"  Precision: {metrics['Precision']:.4f}")
    print(f"  Recall   : {metrics['Recall']:.4f}")
    print(f"  F1-score : {metrics['F1-score']:.4f}")



SVM Results:
  Accuracy : 0.5057
  Precision: 0.2529
  Recall   : 0.5000
  F1-score : 0.3359

KNN Results:
  Accuracy : 0.7241
  Precision: 0.7500
  Recall   : 0.7223
  F1-score : 0.7157

Random Forest Results:
  Accuracy : 1.0000
  Precision: 1.0000
  Recall   : 1.0000
  F1-score : 1.0000

Decision Tree Results:
  Accuracy : 1.0000
  Precision: 1.0000
  Recall   : 1.0000
  F1-score : 1.0000

MLP Results:
  Accuracy : 0.4943
  Precision: 0.2471
  Recall   : 0.5000
  F1-score : 0.3308
